In [25]:
import os
from pyprojroot import here
import pandas as pd
import chromadb
# from openai import AzureOpenAI
from langchain_openai import AzureOpenAIEmbeddings
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv
print(load_dotenv())

True


In [16]:
azure_openai_api_key = os.environ["OPENAI_API_KEY"]
# azure_openai_endpoint = os.environ["OPENAI_API_BASE"]
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://abhil-m5cqrkiy-eastus2.openai.azure.com/"

In [3]:
from langchain_openai import AzureChatOpenAI

model_name = 'gpt-35-turbo-16k'
azure_openai_api_key = os.getenv("OPENAI_API_KEY")
azure_openai_endpoint = "https://abhil-m5cqrkiy-eastus2.openai.azure.com/"

In [6]:
import getpass
import os

if not os.getenv("AZURE_OPENAI_API_KEY"):
    os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass(
        "Enter your AzureOpenAI API key: "
    )

In [ ]:
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://abhil-m5cqrkiy-eastus2.openai.azure.com/"

In [40]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai

In [27]:
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')

In [46]:
# azure_client = AzureOpenAI(
#   api_key = azure_openai_api_key,  
#   api_version = "2024-05-01-preview",
#   azure_endpoint = azure_openai_endpoint
# )
from langchain_openai import AzureOpenAIEmbeddings
google_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
# embeddings = AzureOpenAIEmbeddings(
#     model="text-embedding-3-large",
#     # dimensions: Optional[int] = None, # Can specify dimensions with new text-embedding-3 models
#     # azure_endpoint="https://<your-endpoint>.openai.azure.com/", If not provided, will read env variable AZURE_OPENAI_ENDPOINT
#     api_key='AcCLEewBwXhsYbDe41K4jUEMtX75ipkkH25WkcDOATkq0peNCfdyJQQJ99ALACHYHv6XJ3w3AAAAACOG3TBu',# Can provide an API key directly. If missing read env variable AZURE_OPENAI_API_KEY
#     openai_api_version='2024-05-01-preview', # If not provided, will read env variable AZURE_OPENAI_API_VERSION
# )
chroma_client = chromadb.PersistentClient(path=str(here("data/chroma")))

**Create a collection for data injection**

Throws an error if the table already exists

In [31]:
collection = chroma_client.create_collection(name="titanic_smalll")

In [32]:
file_dir = here("data/for_upload/titanic_small.csv")
df = pd.read_csv(file_dir, nrows=5)

In [33]:
df

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35,0,0,8.0500


NOTE: Process in chunks if dataset is big.

In [47]:
# docs = []
# metadatas = []
# ids = []
# embeddings = []
# for index, row in df.iterrows():
#     output_str = ""
#     # Treat each row as a separate chunk
#     for col in df.columns:
#         output_str += f"{col}: {row[col]},\n"
#     response = azure_client.embeddings.create(
#         input = output_str,
#         model= "text-embedding-ada-002"
#     )
#     embeddings.append(response.data[0].embedding)
#     docs.append(output_str)
#     metadatas.append({"source": "titanic_small"})
#     ids.append(f"id{index}")

# Lists for storing processed data
docs = []
metadatas = []
ids = []
embeddings = []

# Process the DataFrame
for index, row in df.iterrows():
    output_str = ""
    for col in df.columns:
        output_str += f"{col}: {row[col]},\n"
    
    # Generate embeddings using GoogleGenerativeAIEmbeddings
    embedding = google_embeddings.embed_query(output_str)  # Use the `embed_query` method
    embeddings.append(embedding)
    
    # Collect other data
    docs.append(output_str)
    metadatas.append({"source": "titanic_small"})
    ids.append(f"id{index}")

print("Embeddings generated successfully!")

Embeddings generated successfully!


In [48]:
docs

['Survived: 0,\nPclass: 3,\nName: Mr. Owen Harris Braund,\nSex: male,\nAge: 22,\nSiblings/Spouses Aboard: 1,\nParents/Children Aboard: 0,\nFare: 7.25,\n',
 'Survived: 1,\nPclass: 1,\nName: Mrs. John Bradley (Florence Briggs Thayer) Cumings,\nSex: female,\nAge: 38,\nSiblings/Spouses Aboard: 1,\nParents/Children Aboard: 0,\nFare: 71.2833,\n',
 'Survived: 1,\nPclass: 3,\nName: Miss. Laina Heikkinen,\nSex: female,\nAge: 26,\nSiblings/Spouses Aboard: 0,\nParents/Children Aboard: 0,\nFare: 7.925,\n',
 'Survived: 1,\nPclass: 1,\nName: Mrs. Jacques Heath (Lily May Peel) Futrelle,\nSex: female,\nAge: 35,\nSiblings/Spouses Aboard: 1,\nParents/Children Aboard: 0,\nFare: 53.1,\n',
 'Survived: 0,\nPclass: 3,\nName: Mr. William Henry Allen,\nSex: male,\nAge: 35,\nSiblings/Spouses Aboard: 0,\nParents/Children Aboard: 0,\nFare: 8.05,\n']

In [49]:
print(metadatas)
print(ids)

[{'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}, {'source': 'titanic_small'}]
['id0', 'id1', 'id2', 'id3', 'id4']


In [50]:
embeddings[0][:10]

[0.007262978237122297,
 -0.04996849223971367,
 -0.028471125289797783,
 0.02555975690484047,
 0.06109115853905678,
 0.020476624369621277,
 0.015086683444678783,
 0.008503824472427368,
 0.03310805559158325,
 0.005286351311951876]

In [51]:
collection.add(
    documents=docs,
    metadatas=metadatas,
    embeddings=embeddings,
    ids=ids
)

Verify the vectorDB creation

In [52]:
print("Number of vectors in vectordb:", collection.count())

Number of vectors in vectordb: 5


### RAG

In [53]:
from langchain_openai import AzureChatOpenAI

model_name = 'gpt-35-turbo-16k'
azure_openai_api_key = os.getenv("OPENAI_API_KEY")
azure_openai_endpoint = "https://abhil-m5cqrkiy-eastus2.openai.azure.com/"

In [55]:
from openai import AzureOpenAI

In [56]:

model_name = "gpt-35-turbo"
azure_openai_api_key = os.environ["OPENAI_API_KEY"]
azure_openai_endpoint = os.environ["OPENAI_API_BASE"]

In [60]:
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://abhil-m5cqrkiy-eastus2.openai.azure.com/"
azure_client = AzureOpenAI(
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("OPENAI_API_BASE"),
    api_key=os.getenv("OPENAI_API_KEY"),
)

**Perform similarity search**

In [ ]:
query_texts = "what's the average age of survivors"
# response = azure_client.embeddings.create(
#         input = query_texts,
#         model= "text-embedding-ada-002"
#     )
response = google_embeddings.embed_query(
        query_texts
    )
# query_embeddings = response.data[0].embedding
query_embeddings = response
query_embeddings

[0.00730901351198554,
 -0.028869055211544037,
 -0.04360585659742355,
 -0.02102619968354702,
 0.07946974039077759,
 -0.008019681088626385,
 -0.008687163703143597,
 0.03499245271086693,
 0.014832835644483566,
 -0.02934938669204712,
 -0.02888510935008526,
 0.020482564345002174,
 0.012086831964552402,
 0.03439514338970184,
 0.001868960913270712,
 -0.014936462976038456,
 0.021334847435355186,
 0.02827759087085724,
 0.007187835872173309,
 -0.0009289763402193785,
 -0.0014718234306201339,
 0.025732355192303658,
 -0.02884458564221859,
 0.002026940695941448,
 -0.01757277548313141,
 -0.012561892159283161,
 0.06659986078739166,
 -0.08217144757509232,
 -0.006438251119107008,
 0.0778362900018692,
 -0.047188904136419296,
 0.04709474369883537,
 0.0057336799800395966,
 -0.023672834038734436,
 0.00040386177715845406,
 -0.03701629117131233,
 0.03596685454249382,
 -0.006807699799537659,
 0.005301584489643574,
 0.043254051357507706,
 -0.004652557894587517,
 0.014611894264817238,
 -0.06066843867301941,
 -0.

**Load the chromaDB collection for vector search**

In [9]:
vectordb = chroma_client.get_collection(name="titanic_small")
vectordb.count()

30

In [19]:
results = vectordb.query(
    query_embeddings = query_embeddings,
    n_results=1 #top_k
)

results

{'ids': [['id13']],
 'distances': [[0.46567865172602374]],
 'metadatas': [[{'source': 'titanic_small'}]],
 'embeddings': None,
 'documents': [['Survived: 0,\nPclass: 3,\nName: Mr. Anders Johan Andersson,\nSex: male,\nAge: 39,\nSiblings/Spouses Aboard: 1,\nParents/Children Aboard: 5,\nFare: 31.275,\n']],
 'uris': None,
 'data': None}

Pass the results to an LLM

In [20]:
system_role = "You will recieve the user's question along with the search results of that question over a database. Give the user the proper answer."
prompt = f"User's question: {query_texts} \n\n Search results:\n {results}"

messages = [
    {"role": "system", "content": str(
        system_role
        )},
    {"role": "user", "content": prompt}
]

In [21]:
response = azure_client.chat.completions.create(
    model=os.getenv("gpt_deployment_name"),
    messages=messages
)

In [22]:
response.choices[0].message.content

'Based on the search results, we can see that the age of one of the passengers in the dataset is 39 years old. However, this information does not necessarily represent the average age of survivors. To calculate the average age of survivors, we would need more data from the database.'

**Fact check**

In [17]:
df

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35,0,0,8.0500
